<a href="https://colab.research.google.com/github/abhijain864/VTON-Intern/blob/main/aboutyou_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Categories are - 
* MEN
  * main-folder
    1. men_t-shirts  ***1  -  10,753***
    1. men_t-shirts  (long sleeves)  ***10,754  -  11,645***
    1. men_sweaters  ***11,646  -  13,092***
    1. men_sweatshirts  ***13,093  -  15,155***
    1. men_polos  ***15,156  -  16,023***
    1. men_vests  ***16,024  -  16,294***
    1. men_shirts  ***16,295  -  18,921***
  * others
    1. men_hoodies ***1  -  2,385***
    2. men_polos  (plain) 2386  -  
    3. men_shirts (plain)
* WOMEN

#Note we haven't done women dresses beyond page = 200. It was kind of enough dresses. Still if needed we can scrape women's dresses from page = 201 till 259

In [ ]:
import time
import requests
from requests.exceptions import ConnectionError
import os
from bs4 import BeautifulSoup
import re
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
}
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Web_scraping/aboutyou/aboutyou_images

Mounted at /content/drive
/content/drive/My Drive/Web_scraping/aboutyou/aboutyou_images


In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver/usr/bin

     |████████████████████████████████| 911kB 6.8MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:11 https://developer.download.nvidia.com/compute/cuda/repo

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
def del_list( ctg , idx_list):
  for i in idx_list:
    os.remove('0/' + ctg + str(i) + '_0')
    os.remove('1/' + ctg + str(i) + '_1')
    os.remove('2/' + ctg + str(i) + '_2')
def delete_imgs_in_range( ctg , start , end):
  for i in range(start,end+1):
    if os.path.exists('0/'+ctg + str(i) + '_0' +'.jpg'):
      os.remove('0/' + ctg + str(i) + '_0' + '.jpg')
    if os.path.exists( '1/' + ctg + str(i) + '_1' +'.jpg'):
      os.remove( '1/'  + ctg + str(i) + '_1' + '.jpg')
    if os.path.exists(  '2/' + ctg + str(i) + '_2' +'.jpg'):
      os.remove(  '2/' + ctg + str(i) + '_2' + '.jpg')  
    print('deleted',i)
# def delete_imgs(sure=False):
#   if sure:
#     for pic in os.listdir('/content/drive/My Drive/Web_scraping/Hackett/Hackett_images'):
#       if re.split(r'\.',pic)[1]=='html':
#         continue
#       else:
#         os.remove(pic)

In [ ]:
def func_description(tag):
  url = 'https://www.aboutyou.de' + tag['href']
  split1 = re.split(r'/',url)[-2] + ' ' +  re.split(r'/',url)[-1]
  # split2 = re.split(r'\.',split1)[0]
  return split1

In [ ]:
def save( s , img_category,img_urls,description):
  global count
  if len(img_urls)<1 :
    print(description , ' this product was returned')
    return
  file_name = img_category + str(count) + '_0'
  with open('0/' + file_name + '.jpg','wb') as f:
    try:
      im = s.get(img_urls[0], headers=HEADERS)
    except ConnectionError as err:
      time.sleep(10) 
      im = s.get(img_urls[0], headers=HEADERS)
    f.write(im.content)   

  # file_name = img_category + str(count) + '_1'
  # with open('1/' + file_name + '.jpg','wb') as f:
  #   try:
  #     im = s.get(img_urls[1], headers=HEADERS)
  #   except ConnectionError as err:
  #     time.sleep(10) 
  #     im = s.get(img_urls[1], headers=HEADERS)
  #   f.write(im.content) 

  file_name = img_category + str(count) + '_2'
  with open('2/' + file_name + '.jpg','wb') as f:
    try:
      im = s.get(img_urls[2], headers=HEADERS)
    except ConnectionError as err:
      time.sleep(10) 
      im = s.get(img_urls[2], headers=HEADERS)
    f.write(im.content) 

  print(img_category + str(count) + '    name is  ' + description)
  count=count+1

In [ ]:
def generate_image_urls(s , tag):
  imgs = []
  main_page_img = tag.find('img')['src'] # This is the image of product on main page
  url = 'https://www.aboutyou.de' + tag['href']

  try:
    r = s.get(url, headers=HEADERS)
  except ConnectionError as err:
    time.sleep(10) 
    r = s.get(url, headers=HEADERS)


  soup = BeautifulSoup(r.text,'html.parser')
  list = soup.find_all('div' , attrs={'data-test-id':'ImageBoxContainer'})
  if len(list)<5:
    return imgs
    # print('this was returned as no_of_imgs less than 5')
  img_on_img_page = list[1].find('img')['src']  # This is the 2nd image on the product's url


  modified = []
  # if main_page_img==img_on_img_page :
  for tag in soup.find_all('div' , attrs={'data-test-id':'ImageBoxContainer'}):
    img = tag.find('img')['src']  
    imgs.append(img)
  images = [imgs[1],imgs[-1],imgs[0]]

  if re.search('width=400&height=534',images[0]):
    temp_cloth = re.sub('width=400&height=534','width=600&height=800',images[0])
    final_cloth = re.sub('&trim=1','',temp_cloth)
    modified.append(final_cloth)
    modified.append(re.sub('width=400','width=600&height=800',images[1]))
    modified.append(re.sub('width=400','width=600&height=800',images[2]))
  # print(modified)
  return modified 

In [ ]:
def scrape(session, url , ctg):
  print('urls_for_this_page_are_being_generated')
  wd.get(url)
  if wd.current_url!= url:
    print('Something fishy, please check the url')
    return 0
  tag_set = {'a'}
  for i in range(0,11):
    soup = BeautifulSoup(wd.page_source, 'html.parser')
    for tag in soup.find_all('a' , attrs={'data-test-id':'ProductTile'}):
      # print('https://www.aboutyou.de' + tag['href'])
      tag_set.add(tag)
    # print('\n\n now next batch starts','\n\n')
    wd.execute_script('window.scrollBy(0, 2750)')
    time.sleep(.3)
  tag_set = tag_set - {'a'}
  print('urls have been generated, now we will start scraping')
  print('total_count_for_this_category_is ' ,len(list(tag_set)),' but many images dont have either cloth image or model image thus they will be discraded')
  for tag in list(tag_set):
    save( session , ctg ,       generate_image_urls(session , tag)     ,   func_description(tag) ) 
  # return list(tag_set)
  return 1

In [ ]:
def scrape_this(session , ctg , start , end , url):
  for i in range(start,end+1):
    # url = re.sub(re.split(r'&',url)[page_in_url_idx],'page='+str(i),url)
    url =   'https://www.aboutyou.de/c/frauen/bekleidung/kleider-20236?' + 'page='+str(i) + '&sort=topseller'
    print('Scraping page number = ',i)
    print(url)
    flag  = scrape( session , url, ctg)
    if flag==0:
      print('Execution stopped')
      break



---



---



In [ ]:
# wd.close()
# wd = webdriver.Chrome('chromedriver',options=chrome_options)

In [ ]:
# count = 1
# scrape_this('men_t-shirts_')

Scraping page number =  1
https://www.aboutyou.de/c/maenner/bekleidung/shirts/t-shirts-140199?quantityPerPack=28224&page=1&sort=topseller
urls_for_this_page_are_being_generated
urls have been generated, now we will start scraping
total_count_for_this_category_is  100  but many images dont have either cloth image or model image thus they will be discraded
men_t-shirts_1    name is  polo-ralph-lauren t-shirt-6719148
men_t-shirts_2    name is  champion-authentic-athletic-apparel shirt-4710131
men_t-shirts_3    name is  tommy-hilfiger shirt-tommy-logo-tee-5084438
men_t-shirts_4    name is  scotch-soda t-shirt-6668859
men_t-shirts_5    name is  dan-fox-apparel shirt-piet-4789846
men_t-shirts_6    name is  calvin-klein-jeans t-shirt-4655021
men_t-shirts_7    name is  brave-soul t-shirt-notex-ws-6763651
men_t-shirts_8    name is  dan-fox-apparel shirt-luke-4789858
men_t-shirts_9    name is  adidas-originals shirt-surreal-summer-6546889
men_t-shirts_10    name is  key-largo t-shirt-mt-jungle-r

OSError: ignored

In [ ]:
# count = 629
# scrape_this('men_t-shirts_')

Scraping page number =  8
https://www.aboutyou.de/c/maenner/bekleidung/shirts/t-shirts-140199?quantityPerPack=28224&page=8&sort=topseller
urls_for_this_page_are_being_generated
urls have been generated, now we will start scraping
total_count_for_this_category_is  100  but many images dont have either cloth image or model image thus they will be discraded
men_t-shirts_629    name is  levi-s t-shirt-sunset-5724595
men_t-shirts_630    name is  mister-tee shirt-electric-planet-6546581
men_t-shirts_631    name is  guess shirt-5578002
men_t-shirts_632    name is  tommy-jeans t-shirt-5146520
men_t-shirts_633    name is  key-largo t-shirt-mit-v-ausschnitt-3320891
men_t-shirts_634    name is  tom-tailor-denim t-shirt-6547866
men_t-shirts_635    name is  jack-jones shirt-coraz-6748600
men_t-shirts_636    name is  jack-jones t-shirt-onewanthem-6662305
men_t-shirts_637    name is  only-sons t-shirt-klop-6752513
men_t-shirts_638    name is  levi-s shirt-6753132
men_t-shirts_639    name is  nike-spo

In [ ]:
# count = 2682
# scrape_this('men_t-shirts_')

Scraping page number =  30
https://www.aboutyou.de/c/maenner/bekleidung/shirts/t-shirts-140199?quantityPerPack=28224&page=30&sort=topseller
urls_for_this_page_are_being_generated
urls have been generated, now we will start scraping
total_count_for_this_category_is  100  but many images dont have either cloth image or model image thus they will be discraded
men_t-shirts_2682    name is  petrol-industries t-shirt-5683351
men_t-shirts_2683    name is  river-island shirt-6505386
men_t-shirts_2684    name is  ltb shirt-bizefa-6617020
men_t-shirts_2685    name is  adidas-originals shirt-5625651
men_t-shirts_2686    name is  barbour-beacon t-shirt-6705662
men_t-shirts_2687    name is  ovs shirt-nasa3-6699839
men_t-shirts_2688    name is  selected-homme shirt-slhregcarlos-5879758
men_t-shirts_2689    name is  dedicated t-shirt-stockholm-snoopy-flags-6643959
men_t-shirts_2690    name is  oakley t-shirt-scott-tinley-6697626
men_t-shirts_2691    name is  ltb t-shirt-bidisa-6719882
men_t-shirts_26

In [ ]:
# count = 3794
# scrape_this('men_t-shirts_')

Scraping page number =  38
https://www.aboutyou.de/c/maenner/bekleidung/shirts/t-shirts-140199?quantityPerPack=28224&page=38&sort=topseller
urls_for_this_page_are_being_generated
urls have been generated, now we will start scraping
total_count_for_this_category_is  100  but many images dont have either cloth image or model image thus they will be discraded
men_t-shirts_3794    name is  burton-menswear-london shirt-6517842
men_t-shirts_3795    name is  fila t-shirt-amparo-5725200
men_t-shirts_3796    name is  anerkjendt t-shirt-rod-6656647
men_t-shirts_3797    name is  dedicated t-shirt-6651177
men_t-shirts_3798    name is  tommy-jeans shirt-6558156
men_t-shirts_3799    name is  fli-papigu t-shirt-yu-still-brothers-6568820
men_t-shirts_3800    name is  fc-st-pauli t-shirt-5113828
men_t-shirts_3801    name is  united-colors-of-benetton shirt-5722617
men_t-shirts_3802    name is  fli-papigu shirt-das-auge-des-hasen-6584389
men_t-shirts_3803    name is  drykorn shirt-carlo-5702638
men_t-sh

KeyboardInterrupt: ignored

In [ ]:
# s = requests.Session()
# count = 6097
# wd = webdriver.Chrome('chromedriver',options=chrome_options)
# scrape_this(s , 'men_t-shirts_')

Scraping page number =  72
https://www.aboutyou.de/c/maenner/bekleidung/shirts/t-shirts-140199?quantityPerPack=28224&page=72&sort=topseller
urls_for_this_page_are_being_generated
urls have been generated, now we will start scraping
total_count_for_this_category_is  100  but many images dont have either cloth image or model image thus they will be discraded
men_t-shirts_6097    name is  rusty-neal t-shirt-5906157
men_t-shirts_6098    name is  finn-flare kurzarmshirt-6727861
men_t-shirts_6099    name is  logoshirt t-shirt-aquaman-4413129
men_t-shirts_6100    name is  mister-tee t-shirt-4644416
men_t-shirts_6101    name is  lacoste shirt-5685715
men_t-shirts_6102    name is  solid rundhalsshirt-calvin-4382165
men_t-shirts_6103    name is  cleptomanicx t-shirt-old-three-d-6693987
men_t-shirts_6104    name is  camel-active t-shirt-5854200
men_t-shirts_6105    name is  logoshirt t-shirt-batman-3478306
men_t-shirts_6106    name is  solid t-shirt-longo-6522030
men_t-shirts_6107    name is  ble

KeyboardInterrupt: ignored

In [ ]:
# s = requests.Session()
# count = 6228
# wd = webdriver.Chrome('chromedriver',options=chrome_options)
# scrape_this(s , 'men_t-shirts_')

Scraping page number =  74
https://www.aboutyou.de/c/maenner/bekleidung/shirts/t-shirts-140199?quantityPerPack=28224&page=74&sort=topseller
urls_for_this_page_are_being_generated
urls have been generated, now we will start scraping
total_count_for_this_category_is  100  but many images dont have either cloth image or model image thus they will be discraded
men_t-shirts_6228    name is  ilhh shirt-5163183
men_t-shirts_6229    name is  chiemsee shirt-4723830
men_t-shirts_6230    name is  mister-tee t-shirt-highway-inn-4644370
men_t-shirts_6231    name is  selected-homme t-shirt-6545691
men_t-shirts_6232    name is  mister-tee t-shirt-cray-4175450
men_t-shirts_6233    name is  trueprodigy t-shirt-seth-4455636
men_t-shirts_6234    name is  mister-tee shirt-i-love-it-4644379
men_t-shirts_6235    name is  superdry t-shirt-5106549
men_t-shirts_6236    name is  mister-tee t-shirt-3886701
men_t-shirts_6237    name is  s-oliver t-shirt-5714474
men_t-shirts_6238    name is  urban-classics t-shirt

In [ ]:
# s = requests.Session()
# count = 7655
# wd = webdriver.Chrome('chromedriver',options=chrome_options)
# scrape_this(s , 'men_t-shirts_')

Scraping page number =  95
https://www.aboutyou.de/c/maenner/bekleidung/shirts/t-shirts-140199?quantityPerPack=28224&page=95&sort=topseller
urls_for_this_page_are_being_generated
urls have been generated, now we will start scraping
total_count_for_this_category_is  100  but many images dont have either cloth image or model image thus they will be discraded
men_t-shirts_7655    name is  tom-tailor t-shirt-5615375
men_t-shirts_7656    name is  lee shirt-6569850
men_t-shirts_7657    name is  s-oliver t-shirt-5664262
men_t-shirts_7658    name is  jack-jones t-shirt-5121977
men_t-shirts_7659    name is  siksilk t-shirt-6677568
men_t-shirts_7660    name is  billabong shirt-austin-5913850
men_t-shirts_7661    name is  solid shirt-phero-5571237
men_t-shirts_7662    name is  pepe-jeans t-shirt-eggo-6546557
men_t-shirts_7663    name is  only-sons t-shirt-5894720
men_t-shirts_7664    name is  superdry t-shirt-4605471
men_t-shirts_7665    name is  brave-soul t-shirt-federerd-6643201
men_t-shirts_7

NameError: ignored

In [ ]:
# s = requests.Session()
# count = 9943
# wd = webdriver.Chrome('chromedriver',options=chrome_options)
# scrape_this(s , 'men_t-shirts_')

Scraping page number =  130
https://www.aboutyou.de/c/maenner/bekleidung/shirts/t-shirts-140199?quantityPerPack=28224&page=130&sort=topseller
urls_for_this_page_are_being_generated
urls have been generated, now we will start scraping
total_count_for_this_category_is  100  but many images dont have either cloth image or model image thus they will be discraded
men_t-shirts_9943    name is  s-oliver t-shirt-5730281
men_t-shirts_9944    name is  jack-jones t-shirt-4867605
men_t-shirts_9945    name is  s-oliver t-shirt-mit-frontprint-6631569
men_t-shirts_9946    name is  tom-tailor-men t-shirt-4758748
men_t-shirts_9947    name is  superdry t-shirt-4656069
men_t-shirts_9948    name is  s-oliver shirt-6666633
men_t-shirts_9949    name is  bruno-banani t-shirt-5211159
men_t-shirts_9950    name is  superdry t-shirt-4896341
men_t-shirts_9951    name is  jack-jones t-shirt-5028478
men_t-shirts_9952    name is  jan-vanderstorm t-shirt-6726761
men_t-shirts_9953    name is  rusty-neal t-shirt-im-zei

In [ ]:
# with requests.Session() as s:
#   count = 10754
#   wd = webdriver.Chrome('chromedriver',options=chrome_options)
#   scrape_this(s , 'men_long-sleeves_',13)

Scraping page number =  1
https://www.aboutyou.de/c/maenner/bekleidung/shirts/langarmshirts-20955?quantityPerPack=28224&page=1&color=38932%2C38925%2C38931%2C38920%2C38923%2C38928%2C38929%2C38926%2C38921%2C38927%2C38930&page=1
urls_for_this_page_are_being_generated
urls have been generated, now we will start scraping
total_count_for_this_category_is  100  but many images dont have either cloth image or model image thus they will be discraded
men_t-shirts_10754    name is  jack-jones shirt-5204924
men_t-shirts_10755    name is  dan-fox-apparel shirt-lino-6717752
men_t-shirts_10756    name is  river-island shirt-japanese-5870345
men_t-shirts_10757    name is  urban-classics sweatshirt-3613466
men_t-shirts_10758    name is  boss-casual shirt-tacks-6718991
men_t-shirts_10759    name is  hugo shirt-dorrison-6722771
men_t-shirts_10760    name is  esprit shirt-5685345
men_t-shirts_10761    name is  dan-fox-apparel shirt-lino-6717771
men_t-shirts_10762    name is  boss-casual shirt-tempest-5900

In [ ]:
count

11646

In [ ]:
# Note that we have removed plain sweaters
# with requests.Session() as s:
#   wd = webdriver.Chrome('chromedriver',options=chrome_options)
#   url = 'https://www.aboutyou.de/c/maenner/bekleidung/pullover-strick/pullover-20950?color=38932%2C38931%2C38925%2C38933%2C38924%2C38928%2C38929%2C38923%2C38926%2C38921%2C38927%2C38930&sort=topseller&pattern=56707%2C56709%2C35008%2C35458%2C35012%2C35457%2C35007%2C56706%2C35255%2C35011%2C35009%2C56708%2C35010%2C35014&page=1'
#   scrape_this(s , 'men_seaters_',14,url)

Scraping page number =  1
https://www.aboutyou.de/c/maenner/bekleidung/pullover-strick/pullover-20950?color=38932%2C38931%2C38925%2C38933%2C38924%2C38928%2C38929%2C38923%2C38926%2C38921%2C38927%2C38930&page=1&pattern=56707%2C56709%2C35008%2C35458%2C35012%2C35457%2C35007%2C56706%2C35255%2C35011%2C35009%2C56708%2C35010%2C35014&page=1
urls_for_this_page_are_being_generated
urls have been generated, now we will start scraping
total_count_for_this_category_is  100  but many images dont have either cloth image or model image thus they will be discraded
men_seaters_11646    name is  tom-tailor pullover-5565867
men_seaters_11647    name is  jack-jones pullover-lincoln-5721419
men_seaters_11648    name is  esprit pullover-5122223
men_seaters_11649    name is  jack-jones pullover-3933200
men_seaters_11650    name is  iriedaily pullover-theodore-summer-5855649
men_seaters_11651    name is  indicode-jeans pullover-kristian-5864276
men_seaters_11652    name is  edc-by-esprit pullover-noos-plated-cn

In [ ]:
# with requests.Session() as s:
#   wd = webdriver.Chrome('chromedriver',options=chrome_options)
#   url = 'https://www.aboutyou.de/c/maenner/bekleidung/pullover-strick/pullover-20950?color=38920&pattern=56707%2C35008%2C56709%2C35458%2C35012%2C35457%2C35007%2C56706%2C35255%2C35009%2C35011%2C35010%2C56708%2C35014&sort=topseller&page=1'
#   scrape_this(s , 'men_seaters_',6,url)

Scraping page number =  1
https://www.aboutyou.de/c/maenner/bekleidung/pullover-strick/pullover-20950?color=38920&page=1&sort=topseller&page=1
urls_for_this_page_are_being_generated
urls have been generated, now we will start scraping
total_count_for_this_category_is  98  but many images dont have either cloth image or model image thus they will be discraded
men_seaters_12618    name is  s-oliver pullover-4873471
men_seaters_12619    name is  jack-jones strickpullover-5349644
men_seaters_12620    name is  jack-jones pullover-6644481
men_seaters_12621    name is  esprit pullover-5881543
men_seaters_12622    name is  only-sons pullover-6550116
men_seaters_12623    name is  tom-tailor-denim pullover-5697545
men_seaters_12624    name is  tom-tailor-denim pullover-5736444
men_seaters_12625    name is  tommy-hilfiger pullover-pima-5023929
men_seaters_12626    name is  g-star-raw pullover-5919221
men_seaters_12627    name is  jack-jones pullover-justin-5297621
men_seaters_12628    name is  se

In [ ]:
# with requests.Session() as s:
#   wd = webdriver.Chrome('chromedriver',options=chrome_options)
#   url = 'https://www.aboutyou.de/c/maenner/bekleidung/sweat/sweatshirts-20963?sort=topseller&color=38932%2C38925%2C38920%2C38923%2C38928%2C38931%2C38921%2C38919%2C38927%2C38926%2C38929%2C38930&page=1'
#   scrape_this(s , 'men_sweatshirts_',30,url)

Scraping page number =  1
https://www.aboutyou.de/c/maenner/bekleidung/sweat/sweatshirts-20963?sort=topseller&page=1&page=1
urls_for_this_page_are_being_generated
urls have been generated, now we will start scraping
total_count_for_this_category_is  98  but many images dont have either cloth image or model image thus they will be discraded
men_sweatshirts_13093    name is  fila sweater-efim-4651398
men_sweatshirts_13094    name is  adidas-originals sweatshirt-4132838
men_sweatshirts_13095    name is  jack-jones sweatshirt-arid-5563710
men_sweatshirts_13096    name is  tommy-jeans sweatshirt-5594496
men_sweatshirts_13097    name is  solid sweatshirt-ricky-5922698
men_sweatshirts_13098    name is  reebok sportsweatshirt-5293519
men_sweatshirts_13099    name is  tommy-jeans sweatshirt-4857141
men_sweatshirts_13100    name is  urban-classics sweatshirt-4681924
men_sweatshirts_13101    name is  wrangler sweatshirt-5915100
men_sweatshirts_13102    name is  urban-classics sweatshirt-5740355
m

# Please note here we have removed the plain polo shirts

In [ ]:
# count = 15156
# with requests.Session() as s:
#   wd = webdriver.Chrome('chromedriver',options=chrome_options)
#   url = 'https://www.aboutyou.de/c/maenner/bekleidung/shirts/poloshirts-20957?color=38932%2C38925%2C38931%2C38920%2C38923%2C38928%2C38929%2C38926%2C38921%2C38919%2C38927%2C38930&pattern=35458%2C56707%2C35008%2C35010%2C56709%2C35457%2C56708%2C35009%2C35014%2C35254%2C35011%2C35013%2C35007%2C56706&quantityPerPack=28224&page=3&sort=topseller'
#   scrape_this(s , 'men_polo-t-shirts_',14,url , 3)

Scraping page number =  1
https://www.aboutyou.de/c/maenner/bekleidung/shirts/poloshirts-20957?color=38932%2C38925%2C38931%2C38920%2C38923%2C38928%2C38929%2C38926%2C38921%2C38919%2C38927%2C38930&pattern=35458%2C56707%2C35008%2C35010%2C56709%2C35457%2C56708%2C35009%2C35014%2C35254%2C35011%2C35013%2C35007%2C56706&quantityPerPack=28224&page=1&sort=topseller
urls_for_this_page_are_being_generated
urls have been generated, now we will start scraping
total_count_for_this_category_is  100  but many images dont have either cloth image or model image thus they will be discraded
men_polo-t-shirts_15156    name is  boss-athleisure shirt-paule-6526790
men_polo-t-shirts_15157    name is  s-oliver poloshirt-6579993
men_polo-t-shirts_15158    name is  shiwi poloshirt-bart-6771760
men_polo-t-shirts_15159    name is  boss-athleisure shirt-pavel-6526803
men_polo-t-shirts_15160    name is  jack-jones poloshirt-5890342
men_polo-t-shirts_15161    name is  polo-ralph-lauren poloshirt-6553704
men_polo-t-shir

In [ ]:
url = 'https://www.aboutyou.de/c/maenner/bekleidung/shirts/tank-tops-20325?color=38932%2C38925%2C38931%2C38920%2C38923%2C38928%2C38929%2C38926%2C38921%2C38919%2C38927%2C38930&quantityPerPack=28224&page=2&sort=topseller'
with requests.Session() as s:
  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  scrape_this(s , 'men_vests_',1,5,url,2)

Scraping page number =  1
https://www.aboutyou.de/c/maenner/bekleidung/shirts/tank-tops-20325?color=38932%2C38925%2C38931%2C38920%2C38923%2C38928%2C38929%2C38926%2C38921%2C38919%2C38927%2C38930&quantityPerPack=28224&page=1&sort=topseller
urls_for_this_page_are_being_generated
urls have been generated, now we will start scraping
total_count_for_this_category_is  100  but many images dont have either cloth image or model image thus they will be discraded
men_vests_16024    name is  iriedaily top-daily-flag-6611783
men_vests_16025    name is  urban-classics shirt-5738175
men_vests_16026    name is  iriedaily top-monte-noe-5894183
men_vests_16027    name is  solid top-6634935
men_vests_16028    name is  blend top-6757013
men_vests_16029    name is  superdry top-6599192
men_vests_16030    name is  hollister top-6604846
men_vests_16031    name is  camp-david tanktop-mit-logo-artworks-5923293
men_vests_16032    name is  dan-fox-apparel top-nick-6699029
men_vests_16033    name is  solid tankto

#Last count in the main folder currently is 16294

In [ ]:
count = 1
%cd /content/drive/My Drive/Web_scraping/aboutyou/aboutyou_images/others
with requests.Session() as s:
  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  url = 'https://www.aboutyou.de/c/maenner/bekleidung/sweat/hoodies-20964?color=38932%2C38925%2C38931%2C38920%2C38923%2C38928%2C38929%2C38926%2C38921%2C38927%2C38930&page=4&sort=topseller'
  scrape_this(s , 'men_hoodies_',1,33,url,1)

/content/drive/My Drive/Web_scraping/aboutyou/aboutyou_images/others
Scraping page number =  1
https://www.aboutyou.de/c/maenner/bekleidung/sweat/hoodies-20964?color=38932%2C38925%2C38931%2C38920%2C38923%2C38928%2C38929%2C38926%2C38921%2C38927%2C38930&page=1&sort=topseller
urls_for_this_page_are_being_generated
urls have been generated, now we will start scraping
total_count_for_this_category_is  100  but many images dont have either cloth image or model image thus they will be discraded
men_hoodies_1    name is  ragwear sweatshirt-6582780
men_hoodies_2    name is  iriedaily sweatshirt-ninetynine-6569804
men_hoodies_3    name is  boss-casual sweatshirt-safa-russell-athletic-6549697
men_hoodies_4    name is  polo-ralph-lauren sweatshirt-5790150
men_hoodies_5    name is  nike-sportswear sweatshirt-5597903
men_hoodies_6    name is  nike sportsweatshirt-pro-npc-5041394
men_hoodies_7    name is  greenbomb sweatshirt-bike-cyclist-6621312
men_hoodies_8    name is  weekday sweatshirt-6715035
m

In [ ]:
count = 16295
%cd /content/drive/My Drive/Web_scraping/aboutyou/aboutyou_images
with requests.Session() as s:
  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  url = 'https://www.aboutyou.de/c/maenner/bekleidung/hemden-20319?color=38932%2C38925%2C38931%2C38923%2C38928%2C38929%2C38926%2C38921%2C38927%2C38930%2C38924%2C38920&pattern=35007%2C35008%2C35010%2C35009%2C56707%2C35457%2C35458%2C35013%2C56709%2C35011%2C56708%2C35255%2C56706%2C35254%2C35014&sort=topseller&page=2'
  scrape_this(s , 'men_shirts_',1,43,url,3)

/content/drive/My Drive/Web_scraping/aboutyou/aboutyou_images
Scraping page number =  1
https://www.aboutyou.de/c/maenner/bekleidung/hemden-20319?color=38932%2C38925%2C38931%2C38923%2C38928%2C38929%2C38926%2C38921%2C38927%2C38930%2C38924%2C38920&pattern=35007%2C35008%2C35010%2C35009%2C56707%2C35457%2C35458%2C35013%2C56709%2C35011%2C56708%2C35255%2C56706%2C35254%2C35014&sort=topseller&page=1
urls_for_this_page_are_being_generated
urls have been generated, now we will start scraping
total_count_for_this_category_is  100  but many images dont have either cloth image or model image thus they will be discraded
men_shirts_16295    name is  tom-tailor-denim hemd-5616675
men_shirts_16296    name is  camel-active hemd-6546477
men_shirts_16297    name is  tom-tailor hemd-6636766
men_shirts_16298    name is  ben-sherman hemd-6773288
men_shirts_16299    name is  superdry hawaiihemd-6584981
men_shirts_16300    name is  urban-classics hemd-3741384
men_shirts_16301    name is  tom-tailor-denim hemd-5

In [ ]:
count = 18516
%cd /content/drive/My Drive/Web_scraping/aboutyou/aboutyou_images
with requests.Session() as s:
  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  url = 'https://www.aboutyou.de/c/maenner/bekleidung/hemden-20319?color=38932%2C38925%2C38931%2C38923%2C38928%2C38929%2C38926%2C38921%2C38927%2C38930%2C38924%2C38920&pattern=35007%2C35008%2C35010%2C35009%2C56707%2C35457%2C35458%2C35013%2C56709%2C35011%2C56708%2C35255%2C56706%2C35254%2C35014&sort=topseller&page=2'
  scrape_this(s , 'men_shirts_',36,43,url,3)

/content/drive/My Drive/Web_scraping/aboutyou/aboutyou_images
Scraping page number =  36
https://www.aboutyou.de/c/maenner/bekleidung/hemden-20319?color=38932%2C38925%2C38931%2C38923%2C38928%2C38929%2C38926%2C38921%2C38927%2C38930%2C38924%2C38920&pattern=35007%2C35008%2C35010%2C35009%2C56707%2C35457%2C35458%2C35013%2C56709%2C35011%2C56708%2C35255%2C56706%2C35254%2C35014&sort=topseller&page=36
urls_for_this_page_are_being_generated
urls have been generated, now we will start scraping
total_count_for_this_category_is  98  but many images dont have either cloth image or model image thus they will be discraded
men_shirts_18516    name is  only-sons einfarbiges-langarmhemd-4752234
men_shirts_18517    name is  only-sons karo-langarmhemd-5201408
men_shirts_18518    name is  tom-tailor hemd-4699198
men_shirts_18519    name is  iriedaily freizeithemd-mc-vallet-2053757
men_shirts_18520    name is  jack-jones hemd-5216936
men_shirts_18521    name is  s-oliver hemd-4698670
men_shirts_18522    name

In [ ]:
re.sub(r'page=2','page='+str(7),'https://www.aboutyou.de/c/frauen/bekleidung/kleider-20236?page=2&sort=topseller')

'https://www.aboutyou.de/c/frauen/bekleidung/kleider-20236?page=7&sort=topseller'

In [ ]:
re.sub(r'')

In [ ]:
# %cd /content/drive/My Drive/Web_scraping/aboutyou/aboutyou_images/women
# with requests.Session() as s:
#   wd = webdriver.Chrome('chromedriver',options=chrome_options)
#   url = 'https://www.aboutyou.de/c/frauen/bekleidung/kleider-20236?page=1&sort=topseller'
#   scrape_this(s , 'women_dresses_',23,100,url)

/content/drive/My Drive/Web_scraping/aboutyou/aboutyou_images/women
Scraping page number =  23
https://www.aboutyou.de/c/frauen/bekleidung/kleider-20236?page=23&sort=topseller
urls_for_this_page_are_being_generated
urls have been generated, now we will start scraping
total_count_for_this_category_is  100  but many images dont have either cloth image or model image thus they will be discraded
women_dresses_1895    name is  noisy-may kleid-5881230
women_dresses_1896    name is  defacto kleid-6719441
vero-moda-tall kleid-april-6621796  this product was returned
women_dresses_1897    name is  hugo kleid-ermeta-6550471
about-you kleid-polly-4835048  this product was returned
women_dresses_1898    name is  edited kleid-hattie-6533390
pieces kleid-sienna-6576333  this product was returned
women_dresses_1899    name is  ragwear sommerkleid-mike-dress-organic-6619691
women_dresses_1900    name is  tommy-hilfiger kleid-5920240
comma kleid-5854432  this product was returned
vero-moda-tall kleid-h

In [ ]:
count

NameError: ignored

In [ ]:
count = 3484
%cd /content/drive/My Drive/Web_scraping/aboutyou/aboutyou_images/women
with requests.Session() as s:
  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  url = 'https://www.aboutyou.de/c/frauen/bekleidung/kleider-20236?page=1&sort=topseller'
  scrape_this(s , 'women_dresses_',50,100,url)

Streaming output truncated to the last 5000 lines.
women_dresses_3696    name is  finn-flare kleid-6620819
women_dresses_3697    name is  only kleid-winner-4715039
daniel-hechter kleid-cherubini-3750521  this product was returned
women_dresses_3698    name is  set kleid-6769277
women_dresses_3699    name is  basler kleid-bella-vista-5705220
women_dresses_3700    name is  ragwear kleid-tag-4655155
women_dresses_3701    name is  only kleid-winner-6513732
hallhuber kleid-6560191  this product was returned
venice-beach strandkleid-2096796  this product was returned
desires sweatkleid-helena-4385559  this product was returned
set kleid-5673391  this product was returned
venice-beach strandkleid-2096798  this product was returned
women_dresses_3702    name is  cotton-candy sweatkleid-parissa-5754472
women_dresses_3703    name is  set kleid-5618515
kangaroos kleid-5625589  this product was returned
women_dresses_3704    name is  marc-o-polo kleid-6520584
women_dresses_3705    name is  mango k

In [ ]:
count = 6479
%cd /content/drive/My Drive/Web_scraping/aboutyou/aboutyou_images/women
with requests.Session() as s:
  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  url = 'https://www.aboutyou.de/c/frauen/bekleidung/kleider-20236?page=1&sort=topseller'
  scrape_this(s , 'women_dresses_',101,200,url)

Streaming output truncated to the last 5000 lines.
women_dresses_6694    name is  only kleid-eve-5294019
women_dresses_6695    name is  ragwear kleid-dwyn-5725855
women_dresses_6696    name is  ragwear kleid-mascarpone-5909398
women_dresses_6697    name is  paisie kleid-riviera-6577490
women_dresses_6698    name is  lauren-ralph-lauren kleid-kristie-5624062
women_dresses_6699    name is  tommy-jeans kleid-6731445
women_dresses_6700    name is  samsoe-samsoe kleid-elma-5871978
women_dresses_6701    name is  suncoo kleid-candela-6669234
women_dresses_6702    name is  vila kleid-afina-6551430
women_dresses_6703    name is  calvin-klein kleid-6517757
women_dresses_6704    name is  mango kleid-thalia8-6687204
women_dresses_6705    name is  claire kleid-6720294
women_dresses_6706    name is  lauren-ralph-lauren kleid-6613770
women_dresses_6707    name is  10days kleid-6620224
women_dresses_6708    name is  noisy-may kleid-buster-6579260
women_dresses_6709    name is  marc-o-polo-denim kleid-

In [ ]:
count = 10157
%cd /content/drive/My Drive/Web_scraping/aboutyou/aboutyou_images/women
with requests.Session() as s:
  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  url = 'https://www.aboutyou.de/c/frauen/bekleidung/kleider-20236?page=1&sort=topseller'
  scrape_this(s , 'women_dresses_',155,200,url)

/content/drive/My Drive/Web_scraping/aboutyou/aboutyou_images/women
Scraping page number =  155
https://www.aboutyou.de/c/frauen/bekleidung/kleider-20236?page=155&sort=topseller
urls_for_this_page_are_being_generated
urls have been generated, now we will start scraping
total_count_for_this_category_is  99  but many images dont have either cloth image or model image thus they will be discraded
women_dresses_10157    name is  vila kleid-5722889
women_dresses_10158    name is  finn-flare maxikleid-6708412
women_dresses_10159    name is  mango kleid-6777663
women_dresses_10160    name is  green-cotton jerseykleid-4790681
women_dresses_10161    name is  vila kleid-5727480
women_dresses_10162    name is  helmidge sommerkleid-6715244
women_dresses_10163    name is  river-island kleid-5864793
women_dresses_10164    name is  finn-flare sommerkleid-6745023
daniel-hechter kleid-4179398  this product was returned
blend-she kleid-ena-4384008  this product was returned
pieces kleid-5173432  this pro